# Extract Historical Climate Data using FROST API from MET

https://www.met.no/en/free-meteorological-data


best_estimate_mean(air_temperature P3M) 
TAM	Temperature	Homogenised mean temperature (season)	Homogenised seasonal mean temperature. Winter (Dec,Jan,Feb), spring (Mar,Apr,May), summer (Jun,Jul,Aug) and autumn (Sep,Oct,Nov). The mean is an arithmetic mean of daily values.	degC	type: height_above_ground, unit: m, default: 2, values: 2	CF-name

In [1]:
# Libraries needed (pandas is not standard and must be installed in Python)
import requests
import pandas as pd
import yaml
import os 


from pathlib import Path

project_path= Path.cwd().parents[0]
credentials_file = os.path.join(project_path, 'conf/local/credentials.yml')  
print(credentials_file)  

/home/willeke.acampo/git/ac-willeke/urban-climate-services/conf/local/credentials.yml


In [2]:
# read client id from conf/local/credentials.yml
with open(credentials_file, 'r') as f:
    credentials = yaml.safe_load(f)
    client_id = credentials['frost_api']['client_id']
    client_key = credentials['frost_api']['client_key']

### Stations Request

In [3]:
# Define stations endpoint
endpoint_stations = 'https://frost.met.no/locations/v0.jsonld'
# Issue an HTTP GET request
r = requests.get(endpoint_stations, auth=(client_id,''))
# Extract JSON data
json_stations = r.json()

In [4]:
# search for station with name Oslo, Kristiansand, Bodø and Bæerum
for station in json_stations['data']:
    # print station name and coordinates
    if station['name'] == 'Oslo':
        print(station['name'], station['geometry']['coordinates']) 
    if station['name'] == 'Kristiansand':
        print(station['name'], station['geometry']['coordinates'])
    if station['name'] == 'Bodø':
        print(station['name'], station['geometry']['coordinates'])
    if station['name'] == 'Blindern':
        print(station['name'], station['geometry']['coordinates'])


Blindern [10.723072, 59.940625]
Bodø [14.375128, 67.282669]
Kristiansand [7.995733, 58.14615]
Oslo [10.746092, 59.912728]


### Observations Request

In [24]:
# Define endpoint and parameters
endpoint_observations = 'https://frost.met.no/observations/v0.jsonld'

parameters = {
    'sources': 'SN18700,SN39040,SN82310', # oslo/bærum, kristiansand, bodø 
    'elements': 'mean(air_temperature P3M)', # best_estimate_mean(air_temperature P3M) 
    'referencetime': '1900-01-01/2023-10-01',
}
r = requests.get(endpoint_observations, parameters, auth=(client_id,''))
json = r.json()

In [25]:
# Check if the request worked, print out any errors
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])

Data retrieved from frost.met.no!


In [26]:
# This will return a Dataframe with all of the observations in a table format
df = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    df = pd.concat([df, row], ignore_index=True)

df = df.reset_index()

In [27]:
# These additional columns will be kept
columns = ['sourceId','referenceTime','elementId','value','unit','timeOffset']
df2 = df[columns].copy()
# Convert the time value to something Python understands
df2['referenceTime'] = pd.to_datetime(df2['referenceTime'])

In [28]:
# only show the data for the summer months
df_summer = df2[(df2['referenceTime'].dt.month >= 6) & (df2['referenceTime'].dt.month <= 8)]


# Oslo and Bærums Hottest Summer on Record (2018)

- Landsat 8: 2013 - present
- Hottest summer observed at Blindern (Oslo) since 1937 (MET): 1947
- Hottest summer on record Lansat8: 2018

In [29]:
# separate df for oslo
df_oslo = df_summer[df_summer['sourceId'] == 'SN18700:0'].copy()
df_oslo = df_oslo.reset_index()
# sort by warmest summer
df_oslo.sort_values(by=['value'], inplace=True, ascending=False)
display(df_oslo)

,index,sourceId,referenceTime,elementId,value,unit,timeOffset
60,241,SN18700:0,1997-06-01 00:00:00+00:00,mean(air_temperature P3M),18.8,degC,PT0H
81,325,SN18700:0,2018-06-01 00:00:00+00:00,mean(air_temperature P3M),18.8,degC,PT0H
10,41,SN18700:0,1947-06-01 00:00:00+00:00,mean(air_temperature P3M),18.7,degC,PT0H
69,277,SN18700:0,2006-06-01 00:00:00+00:00,mean(air_temperature P3M),18.1,degC,PT0H
18,73,SN18700:0,1955-06-01 00:00:00+00:00,mean(air_temperature P3M),18.1,degC,PT0H
...,...,...,...,...,...,...,...
27,109,SN18700:0,1964-06-01 00:00:00+00:00,mean(air_temperature P3M),14.1,degC,PT0H
28,113,SN18700:0,1965-06-01 00:00:00+00:00,mean(air_temperature P3M),14.1,degC,PT0H
50,201,SN18700:0,1987-06-01 00:00:00+00:00,mean(air_temperature P3M),13.9,degC,PT0H
56,225,SN18700:0,1993-06-01 00:00:00+00:00,mean(air_temperature P3M),13.8,degC,PT0H


# Kristiansand Hottest summer on record  

- Landsat 8: 2013 - present
- Hottest summer observed in Kristiansand since 1937 (MET): 2018
- Hottest summer on record Lansat8: 2018

In [30]:
# separate df for kristiansand
df_kristiansand = df_summer[df_summer['sourceId'] == 'SN39040:0'].copy()
df_kristiansand = df_kristiansand.reset_index()
# sort by warmest summer
df_kristiansand.sort_values(by=['value'], inplace=True, ascending=False)
display(df_kristiansand)


,index,sourceId,referenceTime,elementId,value,unit,timeOffset
77,654,SN39040:0,2018-06-01 00:00:00+00:00,mean(air_temperature P3M),17.5,degC,PT0H
56,571,SN39040:0,1997-06-01 00:00:00+00:00,mean(air_temperature P3M),17.4,degC,PT0H
6,371,SN39040:0,1947-06-01 00:00:00+00:00,mean(air_temperature P3M),17.2,degC,PT0H
62,595,SN39040:0,2003-06-01 00:00:00+00:00,mean(air_temperature P3M),17.0,degC,PT0H
80,666,SN39040:0,2021-06-01 00:00:00+00:00,mean(air_temperature P3M),16.8,degC,PT0H
...,...,...,...,...,...,...,...
23,439,SN39040:0,1964-06-01 00:00:00+00:00,mean(air_temperature P3M),13.7,degC,PT0H
11,391,SN39040:0,1952-06-01 00:00:00+00:00,mean(air_temperature P3M),13.6,degC,PT0H
24,443,SN39040:0,1965-06-01 00:00:00+00:00,mean(air_temperature P3M),13.6,degC,PT0H
46,531,SN39040:0,1987-06-01 00:00:00+00:00,mean(air_temperature P3M),13.4,degC,PT0H


### Bodø Hottest summer on record 

- Landsat 8: 2013 - present
- Hottest summer observed in Bodø since 1937 (MET): 2023
- Hottest summer on record Lansat8: 2023

In [31]:
df_summer

,sourceId,referenceTime,elementId,value,unit,timeOffset
1,SN18700:0,1937-06-01 00:00:00+00:00,mean(air_temperature P3M),16.9,degC,PT0H
5,SN18700:0,1938-06-01 00:00:00+00:00,mean(air_temperature P3M),15.7,degC,PT0H
9,SN18700:0,1939-06-01 00:00:00+00:00,mean(air_temperature P3M),16.4,degC,PT0H
13,SN18700:0,1940-06-01 00:00:00+00:00,mean(air_temperature P3M),15.9,degC,PT0H
17,SN18700:0,1941-06-01 00:00:00+00:00,mean(air_temperature P3M),16.5,degC,PT0H
...,...,...,...,...,...,...
680,SN82310:0,2018-06-01 00:00:00+00:00,mean(air_temperature P3M),12.6,degC,PT0H
683,SN82310:0,2019-06-01 00:00:00+00:00,mean(air_temperature P3M),13.2,degC,PT0H
687,SN82310:0,2020-06-01 00:00:00+00:00,mean(air_temperature P3M),13.5,degC,PT0H
691,SN82310:0,2022-06-01 00:00:00+00:00,mean(air_temperature P3M),13.0,degC,PT0H


In [32]:
# separate df for bodø
df_bodø = df_summer[df_summer['sourceId'] == 'SN82310:0'].copy()
df_bodø = df_bodø.reset_index()
# sort by warmest summer
df_bodø.sort_values(by=['value'], inplace=True, ascending=False)
display(df_bodø)


,index,sourceId,referenceTime,elementId,value,unit,timeOffset
5,695,SN82310:0,2023-06-01 00:00:00+00:00,mean(air_temperature P3M),14.1,degC,PT0H
3,687,SN82310:0,2020-06-01 00:00:00+00:00,mean(air_temperature P3M),13.5,degC,PT0H
2,683,SN82310:0,2019-06-01 00:00:00+00:00,mean(air_temperature P3M),13.2,degC,PT0H
4,691,SN82310:0,2022-06-01 00:00:00+00:00,mean(air_temperature P3M),13.0,degC,PT0H
0,676,SN82310:0,2017-06-01 00:00:00+00:00,mean(air_temperature P3M),12.9,degC,PT0H
1,680,SN82310:0,2018-06-01 00:00:00+00:00,mean(air_temperature P3M),12.6,degC,PT0H


### Hottest summer Day for Oslo and Bærum (2018)


In [35]:
# Define endpoint and parameters
endpoint_observations = 'https://frost.met.no/observations/v0.jsonld'

parameters = {
    'sources': 'SN18700,SN39040', # oslo/bærum
    'elements': 'mean(air_temperature)', # best_estimate_mean(air_temperature P3M) 
    'referencetime': '2018-06-01/2018-08-31',
}
r = requests.get(endpoint_observations, parameters, auth=(client_id,''))
json = r.json()

In [36]:
# Check if the request worked, print out any errors
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])

Error! Returned status code 412
Message: 412
Reason: No time series found for this combination of parameters, check /observations/availableTimeSeries for more information.
